In [12]:
# Load calibration data
with open('./process_MPIIGaze/batches/all_data_200_100.pkl', 'rb') as f:
    X, Y = pickle.load(f)

In [1]:
from keras.models import load_model
import numpy as np
import cv2
import pickle


In [13]:

# Initialize your gaze detection model
model = load_model('./eye_gaze_v23_1_10.h5')

# Predict gaze points for each image in the calibration dataset
predicted_gaze_points = []
for image in X:
    image_reshaped = np.expand_dims(image, axis=0)
    predicted_point = model.predict(image_reshaped)  
    predicted_gaze_points.append(predicted_point)

# Create the dataset for the adjustment model
adjustment_dataset = {
    'predicted_gaze_points': predicted_gaze_points,
    'actual_gaze_points': Y
}

# Save the dataset into a new pickle file
with open('adjustment_dataset.pkl', 'wb') as f:
    pickle.dump(adjustment_dataset, f)


1/1 [==============================] - 0s 58ms/step


In [14]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import pickle
import numpy as np


In [15]:
# Load your data
with open('./adjustment_dataset.pkl', 'rb') as f:
    calibration_data = pickle.load(f)


X = np.squeeze(np.array(calibration_data['predicted_gaze_points']), axis=1)  # Remove extra dimensions
y = np.array(calibration_data['actual_gaze_points'])[:, :2]  # Remove extra dimensions

In [17]:
len(X), len(y)

(3978, 3978)

In [18]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=None)

In [19]:
X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)
X_val = X_val.astype(np.float32)
y_val = y_val.astype(np.float32)

In [20]:
# Initialize the XGBoost regressor model
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 10, alpha = 10, n_estimators = 100)

# Train the model
xg_reg.fit(X_train, y_train)

# Predict on the validation set
y_pred = xg_reg.predict(X_val)

# Evaluate the model
mse = mean_squared_error(y_val, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 0.009056332521140575


In [21]:
# Save the model to a file
with open('xgboost_adjustment_model.pkl', 'wb') as f:
    pickle.dump(xg_reg, f)

In [ ]:
with open('./adjustment_dataset.pkl', 'rb') as f:
    data = pickle.load(f)
x= data['predicted_gaze_points']
x = np.squeeze(np.array(x))
x.shape

In [ ]:
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

# Load your data
with open('./adjustment_dataset.pkl', 'rb') as f:
    calibration_data = pickle.load(f)


X = np.squeeze(np.array(calibration_data['predicted_gaze_points']), axis=1)  # Remove extra dimensions
y = np.array(calibration_data['actual_gaze_points'])[:, :2]  # Remove extra dimensions
# X = calibration_data['predicted_gaze_points']  # Predicted gaze points from the main model
# y = calibration_data['actual_gaze_points']  # Actual gaze points

print(X.shape)
print(y.shape)

# Split the dataset
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=None)


# Initialize and train the model
model = RandomForestRegressor(n_estimators=100, random_state=None)
model.fit(X_train, y_train)

# Predict on the validation set
y_pred = model.predict(X_val)

# Evaluate the model
mse = mean_squared_error(y_val, y_pred)
print(f'Mean Squared Error: {mse}')

In [ ]:
# Save the model
with open('adjustment_model.pkl', 'wb') as f:
    pickle.dump(model, f)